In [126]:
import pandas as pd

filmes = pd.read_csv("movies.csv")
filmes.columns = ["filmeID", "titulo", "generos"]
filmes = filmes.set_index("filmeID")
filmes.head ()

,titulo,generos
filmeID,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [127]:
notas = pd.read_csv("ratings.csv")
notas.columns = ["usuarioID", "filmeID", "notas", "momento"]
notas.head()

,usuarioID,filmeID,notas,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [128]:
notas.describe()

,usuarioID,filmeID,notas,momento
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


# **Primeira tentativa de recomendação**

In [129]:
totalVotos = notas["filmeID"].value_counts()
totalVotos.head()

356     329
318     317
296     307
593     279
2571    278
Name: filmeID, dtype: int64

In [130]:
filmes['totalVotos'] = totalVotos
filmes.head()

,titulo,generos,totalVotos
filmeID,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,110.0
3,Grumpier Old Men (1995),Comedy|Romance,52.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
5,Father of the Bride Part II (1995),Comedy,49.0


In [131]:
#indicando os filmes a partir do total de votos e visualizações
filmes.sort_values("totalVotos", ascending= False).head(10)

,titulo,generos,totalVotos
filmeID,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0
110,Braveheart (1995),Action|Drama|War,237.0
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0


In [132]:
import pandas
notas_medias = notas.groupby("filmeID").mean()["notas"]
notas_medias.head()

filmeID
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: notas, dtype: float64

In [133]:
filmes["nota_media"] = notas_medias
filmes.sort_values("totalVotos", ascending= False).head(10)

,titulo,generos,totalVotos,nota_media
filmeID,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0,3.750000
110,Braveheart (1995),Action|Drama|War,237.0,4.031646
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0,3.970982


# **Segunda heurística**

In [134]:
filmes.sort_values("nota_media", ascending= False).head(10)
#possivel perceber que por mais que tenham boas notas, tem apenas 1 voto
#com isso pode-se concluir que temos que levar em consideração muito mais aspectos

,titulo,generos,totalVotos,nota_media
filmeID,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0
6201,Lady Jane (1986),Drama|Romance,1.0,5.0
102217,Bill Hicks: Revelations (1993),Comedy,1.0,5.0
102084,Justice League: Doom (2012),Action|Animation|Fantasy,1.0,5.0
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,1.0,5.0


In [135]:
filmes.query("totalVotos >= 10").sort_values("nota_media", 
                                                  ascending= False).head(10)

,titulo,generos,totalVotos,nota_media
filmeID,,,,
1041,Secrets & Lies (1996),Drama,11.0,4.590909
3451,Guess Who's Coming to Dinner (1967),Drama,11.0,4.545455
1178,Paths of Glory (1957),Drama|War,12.0,4.541667
1104,"Streetcar Named Desire, A (1951)",Drama,20.0,4.475000
2360,"Celebration, The (Festen) (1998)",Drama,12.0,4.458333
1217,Ran (1985),Drama|War,15.0,4.433333
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
951,His Girl Friday (1940),Comedy|Romance,14.0,4.392857
1927,All Quiet on the Western Front (1930),Action|Drama|War,10.0,4.350000


In [136]:
filmesMais50Votos = filmes.query("totalVotos >= 50")
filmesMais50Votos.sort_values("nota_media", 
                                                  ascending= False).head(10)

,titulo,generos,totalVotos,nota_media
filmeID,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000


Recomendando a partir do que foi assistido, a partir do que se sabe do cliente

In [137]:
assistidos = [1,21,19,10,11,7,2]
filmes.loc[assistidos]

,titulo,generos,totalVotos,nota_media
filmeID,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
21,Get Shorty (1995),Comedy|Crime|Thriller,89.0,3.494382
19,Ace Ventura: When Nature Calls (1995),Comedy,88.0,2.727273
10,GoldenEye (1995),Action|Adventure|Thriller,132.0,3.496212
11,"American President, The (1995)",Comedy|Drama|Romance,70.0,3.671429
7,Sabrina (1995),Comedy|Romance,54.0,3.185185
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818


In [138]:
adventure_Children_fantasy = filmesMais50Votos.query("generos=='Adventure|Children|Fantasy'")
adventure_Children_fantasy.drop(assistidos, errors='ignore').sort_values("nota_media", ascending= False).head(5)

,titulo,generos,totalVotos,nota_media
filmeID,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,107.0,3.761682
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,62.0,3.443548


# **Procurar usuários similares**
**Distâncias de usuários no dataset**




In [139]:
import numpy as np

def distanciaVetores(a,b):
  return np.linalg.norm(a - b)

In [140]:
#localizando as notas do usuario 1
def notasUsuario(usuarioID):
  notasUsuario = notas.query("usuarioID==%d" % usuarioID)
  notasUsuario = notasUsuario[["filmeID", "notas"]].set_index("filmeID")
  return notasUsuario

In [141]:
notasUsuario(1)

,notas
filmeID,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0
...,...
3744,4.0
3793,5.0
3809,4.0


In [142]:
usuario1 = notasUsuario(1)
usuario4 = notasUsuario(4)


In [143]:
usuario1.head()

,notas
filmeID,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0


In [144]:
usuario4.head()

,notas
filmeID,
21,3.0
32,2.0
45,3.0
47,2.0
52,3.0


In [145]:
#comparando os filmes do u1 e u4, no dropna() exclui o que um usuario assistiu e o outro nao
diferencas = usuario1.join(usuario4, lsuffix="_usuario1", rsuffix="_usuario4").dropna()
distanciaVetores(diferencas['notas_usuario1'], diferencas['notas_usuario4'])

11.135528725660043

In [146]:
def distanciaUsuarios(usuarioId1, usuarioId2):
  notas1 = notasUsuario(usuarioId1)
  notas2 = notasUsuario(usuarioId2)
  diferencas = notas1.join(notas2, lsuffix="_n1", rsuffix="_n2").dropna()
  distancia = distanciaVetores(diferencas['notas_n1'], diferencas['notas_n2'])
  return [usuarioId1, usuarioId2, distancia]

In [147]:
distanciaUsuarios(1,4)

[1, 4, 11.135528725660043]

In [148]:
qtdUsuarios = len(notas['usuarioID'].unique())
print("Temos %d usuarios" % qtdUsuarios)

Temos 610 usuarios


In [149]:
def distanciaTodos(voceID):
  distancias = []
  # exibindo a distancia do usuario 1 e usuario x
  for usuarioID in notas['usuarioID'].unique():
    informacoes = distanciaUsuarios(voceID, usuarioID)
    distancias.append(informacoes)
    return distancias

distanciaTodos(1)[:5]

[[1, 1, 0.0]]

In [150]:
# fazendo o mesmo codigo acima mais simplificado e com um dataframe
def distanciaTodos(voceID):
  todosUsuarios =  notas['usuarioID'].unique()
  distancias = [distanciaUsuarios(voceID, usuarioID)  for usuarioID in todosUsuarios]
  distancias = pd.DataFrame(distancias, columns = ["voce", "outro_usuario", "distancias"])
  return distancias

distanciaTodos(1).head()

,voce,outro_usuario,distancias
0,1,1,0.000000
1,1,2,1.414214
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657


# **Usuários filmes em comum são descartados**

In [151]:
def distanciaUsuarios(usuarioId1, usuarioId2, minimo = 5):
  notas1 = notasUsuario(usuarioId1)
  notas2 = notasUsuario(usuarioId2)
  diferencas = notas1.join(notas2, lsuffix="_n1", rsuffix="_n2").dropna()
  if (len(diferencas) < minimo):
     return [usuarioId1, usuarioId2, 100000]
 
  distancia = distanciaVetores(diferencas['notas_n1'], diferencas['notas_n2'])
  return [usuarioId1, usuarioId2, distancia]

In [152]:
distanciaTodos(1).head()

,voce,outro_usuario,distancias
0,1,1,0.000000
1,1,2,100000.000000
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657


In [153]:
#pessoas mais proximas do usuario 1 a partir das notas atribuidas aos filmes (minimo de 5 filmes)
def maisProximosde(voce):
  distancias = distanciaTodos(voce)
  distancias = distancias.sort_values("distancias")
  distancias = distancias.set_index("outro_usuario").drop(voce)
  return distancias

In [154]:
#pessoas mais proximas do usuario 1 e a distancia:
maisProximosde(1)

,voce,distancias
outro_usuario,,
77,1,0.000000
511,1,0.500000
366,1,0.707107
523,1,1.000000
49,1,1.000000
...,...,...
190,1,100000.000000
60,1,100000.000000
576,1,100000.000000


# **Parametros para teste**

In [155]:
def maisProximosde(voce, n = None):
  distancias = distanciaTodos(voceID, n = n)
  distancias = distancias.sort_values("distancias")
  distancias = distancias.set_index("outro_usuario").drop(voceID)
  return distancias

In [156]:
def maisProximosde(voce, n = None):
  TodosUsuarios = notas['usuarioID'].unique()
  if n:
    TodosUsuarios = TodosUsuarios[:n]
  distancias = [distanciaUsuarios(voce, usuarioID) for usuarioID in TodosUsuarios]
  distancias = pd.DataFrame(distancias, columns = ["voce", "outro_usuario", "distancias"])
  return distancias

In [157]:
#mais proximos de 1, os 50 primeiros usuarios a serem analisados
maisProximosde(1, n = 50)

,voce,outro_usuario,distancias
0,1,1,0.000000
1,1,2,100000.000000
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657
5,1,6,8.602325
6,1,7,7.106335
7,1,8,3.741657
8,1,9,1.000000
9,1,10,7.858117


In [158]:
def distanciaUsuarios(usuarioId1, usuarioId2, minimo = 5):
  notas1 = notasUsuario(usuarioId1)
  notas2 = notasUsuario(usuarioId2)
  diferencas = notas1.join(notas2, lsuffix="_n1", rsuffix="_n2").dropna()
  if (len(diferencas) < minimo):
     return None
 
  distancia = distanciaVetores(diferencas['notas_n1'], diferencas['notas_n2'])
  return [usuarioId1, usuarioId2, distancia]

In [159]:
def maisProximosde(voce, n = None):
  TodosUsuarios = notas['usuarioID'].unique()
  if n:
    TodosUsuarios = TodosUsuarios[:n]
  distancias = [distanciaUsuarios(voce, usuarioID) for usuarioID in TodosUsuarios]
  distancias = list(filter(None, distancias))
  distancias = pd.DataFrame(distancias, columns = ["voce", "outro_usuario", "distancias"])
  return distancias

In [160]:
#mais proximos de 1, os 50 primeiros usuarios a serem analisados, mas não os 50 mais proximos
# n : numeros de usuarios a analisar

maisProximosde(1, n = 50)

,voce,outro_usuario,distancias
0,1,1,0.000000
1,1,3,8.200610
2,1,4,11.135529
3,1,5,3.741657
4,1,6,8.602325
5,1,7,7.106335
6,1,8,3.741657
7,1,9,1.000000
8,1,10,7.858117
9,1,11,4.242641


In [161]:
# n : numeros de usuarios a analisar
def sugerePara(voce, n = None):
  notasdeVoce = notasUsuario(voce)
  filmesqueVocejaviu = notasdeVoce.index

  similares = maisProximosde(voce, n = n)
  similar = similares.iloc[0].name
  notasDoSimilar = notasUsuario(similar)
  notasDoSimilar = notasDoSimilar.drop(filmesqueVocejaviu, errors='ignore')
  recomendacoes = notasDoSimilar.sort_values("notas", ascending = False)
  return recomendacoes.join(filmes)

In [162]:
sugerePara(1).head()

,notas,titulo,generos,totalVotos,nota_media
filmeID,,,,,


# **Sugerindo baseado em vários usuários**

In [173]:
def maisProximosde(voceID, n_maisProximos = 10, n = None):
    distancias = distanciaTodos(voceID, n = n)
    distancias = distancias.sort_values("distancias")
    distancias = distancias.set_index("outro_usuario").drop(voceID)
    return distancias.head(n_maisProximos)

In [ ]:
maisProximosde(1, n=300)

In [178]:
# n : numeros de usuarios a analisar
def sugerePara(voce, n_maisProximos=10, n= None):
  notasdeVoce = notasUsuario(voce)
  filmesqueVocejaviu = notasdeVoce.index

  similares = maisProximosde(voce, n_maisProximos = n_maisProximos, n= n)
  usuarioSimilares = similares.index
  notasDoSimilar = notas.set_index("usuarioID").loc[usuarioSimilares]
  recomendacoes = notasDoSimilar.groupby("filmeID").loc[usuarioSimilares]
  recomendacoes = notasDoSimilar.sort_values("notas", ascending = False)
  return recomendacoes.join(filmes)

In [ ]:
sugerePara(1, n = 50).head()

esse conceito de pegar os usuários mais próximos para analisar é o que define o método Knn - K-nearest neighbors (k vizinhos proximos). Portanto abaixo a função está renomeada

In [180]:
def knn(voceID, k_maisProximos = 10, n = None):
    distancias = distanciaTodos(voceID, n = n)
    distancias = distancias.sort_values("distancias")
    distancias = distancias.set_index("outro_usuario").drop(voceID)
    return distancias.head(k_maisProximos)

In [181]:
def sugerePara(voce, k_maisProximos=10, n= None):
  notasdeVoce = notasUsuario(voce)
  filmesqueVocejaviu = notasdeVoce.index

  similares = maisProximosde(voce, k_maisProximos = k_maisProximos, n= n)
  usuarioSimilares = similares.index
  notasDoSimilar = notas.set_index("usuarioID").loc[usuarioSimilares]
  recomendacoes = notasDoSimilar.groupby("filmeID").loc[usuarioSimilares]
  recomendacoes = notasDoSimilar.sort_values("notas", ascending = False)
  return recomendacoes.join(filmes)